In [1]:
# 패키지 불러오기
# ===== 기본 라이브러리 =====
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime
import dask.dataframe as dd
from multiprocessing import Pool, cpu_count
from math import ceil

# ===== 시각화 =====
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

FONT_PATH = r"C:\Windows\Fonts\malgun.ttf"
FONT = fm.FontProperties(fname=FONT_PATH)


# ===== 출력 옵션 =====
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None) # 50
pd.set_option("display.max_colwidth", None)  # 컬럼 내용도 잘리지 않게
pd.set_option("display.width", 1000)

# ===== 시각화 스타일 =====
sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

In [3]:
# =========================
# 가용재고 데이터
# =========================
import os
import time
import pandas as pd

path = 'W:/공용 드라이브/02. 재고/01. 재고현황 (일자별)/2025/'
save_path = r'C:/Users/user04/code/reserve_stock/reserve/available_stock_parts/'
os.makedirs(save_path, exist_ok=True)

target_files = []
t0 = time.time()
print('▶ 1️⃣ 가용재고 파일 수집 시작 (6월부터)')

# 1️⃣ 대상 파일 수집 (폴더 기준 MMDD, 6월부터)
for root, _, files in os.walk(path):
    date = os.path.basename(root)  # 예: '0601', '0715'

    if not (len(date) == 4 and date.isdigit()):
        continue

    folder_month = int(date[:2])
    if folder_month < 6:
        continue  # 6월 이전 폴더 제외

    for f in files:
        if (
            '타점현재고현황_상세정보' in f and
            f.strip().replace(' ', '').lower().endswith('.xlsx')
        ):
            try:
                digits = ''.join(filter(str.isdigit, f))
                file_ym = digits[:6]        # YYYYMM
                file_month = int(file_ym[4:6])
            except Exception:
                continue

            if file_month < 6:
                continue

            full_path = os.path.join(root, f)
            target_files.append((full_path, date))

# 날짜 정렬
target_files.sort(key=lambda x: x[1])

print(f'✔ 파일 수집 완료: {len(target_files)} files '
      f'({(time.time() - t0)/60:.1f}분 소요)')

if not target_files:
    raise ValueError('❌ 가용재고 파일이 조건에 맞게 하나도 없습니다')

# 2️⃣ 파일별 로딩 및 저장
print('▶ 2️⃣ 파일 로딩 및 개별 저장 시작')

for i, (file_path, date) in enumerate(target_files, start=1):
    part_file = os.path.join(save_path, f'available_stock_{date}.parquet')

    if os.path.exists(part_file):
        print(f'⏩ [{i}] {date}: 이미 저장됨 → 건너뜀')
        continue

    try:
        print(f'[{i}] {date}: 로딩 중 → {os.path.basename(file_path)}')
        df = pd.read_excel(file_path)

        # 컬럼 정규화
        df.columns = (
            df.columns
            .astype(str)
            .str.strip()
            .str.replace(' ', '', regex=False)
        )

        df['기준일자'] = date

        # 저장
        df.to_parquet(part_file, engine='pyarrow', index=False)
        print(f'✔ 저장 완료: {part_file}')

    except Exception as e:
        print(f'❌ 예외 발생 → {file_path} → {e}')

print(f'✔ 파일 로딩 및 저장 완료 '
      f'({(time.time() - t0)/60:.1f}분 경과)')


▶ 1️⃣ 가용재고 파일 수집 시작 (6월부터)
✔ 파일 수집 완료: 209 files (0.4분 소요)
▶ 2️⃣ 파일 로딩 및 개별 저장 시작
⏩ [1] 0601: 이미 저장됨 → 건너뜀
⏩ [2] 0602: 이미 저장됨 → 건너뜀
⏩ [3] 0603: 이미 저장됨 → 건너뜀
⏩ [4] 0604: 이미 저장됨 → 건너뜀
⏩ [5] 0605: 이미 저장됨 → 건너뜀
⏩ [6] 0606: 이미 저장됨 → 건너뜀
⏩ [7] 0607: 이미 저장됨 → 건너뜀
⏩ [8] 0608: 이미 저장됨 → 건너뜀
⏩ [9] 0609: 이미 저장됨 → 건너뜀
⏩ [10] 0610: 이미 저장됨 → 건너뜀
⏩ [11] 0611: 이미 저장됨 → 건너뜀
⏩ [12] 0612: 이미 저장됨 → 건너뜀
⏩ [13] 0613: 이미 저장됨 → 건너뜀
⏩ [14] 0614: 이미 저장됨 → 건너뜀
⏩ [15] 0615: 이미 저장됨 → 건너뜀
⏩ [16] 0616: 이미 저장됨 → 건너뜀
⏩ [17] 0617: 이미 저장됨 → 건너뜀
⏩ [18] 0618: 이미 저장됨 → 건너뜀
⏩ [19] 0619: 이미 저장됨 → 건너뜀
⏩ [20] 0620: 이미 저장됨 → 건너뜀
⏩ [21] 0621: 이미 저장됨 → 건너뜀
⏩ [22] 0622: 이미 저장됨 → 건너뜀
⏩ [23] 0623: 이미 저장됨 → 건너뜀
⏩ [24] 0624: 이미 저장됨 → 건너뜀
⏩ [25] 0625: 이미 저장됨 → 건너뜀
⏩ [26] 0626: 이미 저장됨 → 건너뜀
⏩ [27] 0627: 이미 저장됨 → 건너뜀
⏩ [28] 0628: 이미 저장됨 → 건너뜀
⏩ [29] 0629: 이미 저장됨 → 건너뜀
⏩ [30] 0630: 이미 저장됨 → 건너뜀
⏩ [31] 0701: 이미 저장됨 → 건너뜀
⏩ [32] 0702: 이미 저장됨 → 건너뜀
⏩ [33] 0703: 이미 저장됨 → 건너뜀
⏩ [34] 0704: 이미 저장됨 → 건너뜀
⏩ [35] 0705: 이미 저장됨 → 건너뜀
⏩ [36] 0706: 이미 저

In [ ]:
import duckdb

save_path = "C:/Users/user04/code/reserve_stock/reserve/available_stock_parts"
final_save_path = "C:/Users/user04/code/reserve_stock/reserve/stock(2506_2512).parquet"

con = duckdb.connect()

con.execute(f"""
    COPY (
        SELECT * FROM parquet_scan('{save_path}/available_stock_*.parquet', union_by_name=True)
    )
    TO '{final_save_path}' (FORMAT 'parquet');
""")

print("최종 병합 및 저장 완료 (DuckDB)")